# API Requête_ Fonctionnel

In [4]:
# on importe les librairies
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time # on importe le time pour faire des pauses vis à vis de l'api qui nous donne un rate limit


# on créé une connexion avec les données du compte.
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='fd7b3fa38eba45f090afa8c8f208f799',

                                               client_secret='89435de644764ccdaf7eb92e3d006c5d',

                                               redirect_uri='http://localhost:8080',

                                               scope='user-library-read'))

### 1_ Requête artistes+ID

In [4]:
# récupération des données via l'api

# nous avons besoin de deux listes pour récupérer les noms et les id spotify
df_list_artist = pd.DataFrame(columns=['artist_name','artist_id'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_artist_year( year):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    artist_name = []
    id_artist_spotify = []
    
    # on est limité à 1000 retours max que l'on fait à travers des pages de 50 retours via un for
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        track_results = sp.search(q='year:'+str(year), type='album', limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(track_results['albums']['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/search
            artist_name.append(t['artists'][0]['name'])
            id_artist_spotify.append(t['artists'][0]['id'])
    
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name' : artist_name, 'artist_id' : id_artist_spotify})
    df_artist.drop_duplicates(inplace=True)
    df_list_artist2 = pd.concat([df_list_artist, df_artist])
    
    # on affiche un petit message pour chaque année récupérée
    print(f'Année {year} effectuée ! avec {df_artist.shape[0]} lignes sur l"année et {df_list_artist2.shape[0]} lignes sur la liste')
    return df_list_artist2

In [8]:
for year in range(1920, 2026, 1):
    df_list_artist = search_artist_year(year)
    df_list_artist.drop_duplicates(inplace=True)

Année 1920 effectuée ! avec 201 lignes sur l"année et 13544 lignes sur la liste
Année 1921 effectuée ! avec 179 lignes sur l"année et 13522 lignes sur la liste
Année 1922 effectuée ! avec 125 lignes sur l"année et 13468 lignes sur la liste
Année 1923 effectuée ! avec 144 lignes sur l"année et 13487 lignes sur la liste
Année 1924 effectuée ! avec 589 lignes sur l"année et 13932 lignes sur la liste
Année 1925 effectuée ! avec 241 lignes sur l"année et 13604 lignes sur la liste
Année 1926 effectuée ! avec 263 lignes sur l"année et 13626 lignes sur la liste
Année 1927 effectuée ! avec 276 lignes sur l"année et 13645 lignes sur la liste
Année 1928 effectuée ! avec 232 lignes sur l"année et 13601 lignes sur la liste
Année 1929 effectuée ! avec 126 lignes sur l"année et 13495 lignes sur la liste
Année 1930 effectuée ! avec 135 lignes sur l"année et 13504 lignes sur la liste
Année 1931 effectuée ! avec 105 lignes sur l"année et 13474 lignes sur la liste
Année 1932 effectuée ! avec 93 lignes su

### 2_ Requête albums depuis liste artistes

In [5]:
# pour tester, on récupère le df créé juste au dessus en utilisant la première cellule
data = pd.read_csv('df_list_artist_p1.csv')

In [6]:
data1 = data[0:100]

In [2]:
def search_album_artist(artist_id):
    
    artist_sp_id = []
    artist_name = []
    album_name = []
    album_id = []
    release_date = []
    total_tracks = []
    url_image = []

    offset = 0  # initialisation offset
    limit = 50  # nb max albums par requête

    print(f"Récupération des albums pour l'artiste {artist_id}")

    while True:
        albums_results = sp.artist_albums(artist_id, include_groups='album', limit=limit, offset=offset)

        if not albums_results['items']:
            break  # arrêt si plus d'albums

        for t in albums_results['items']:
            
            artist_sp_id.append(artist_id)
            artist_name.append(t['artists'][0]['name'])
            album_name.append(t['name'])
            album_id.append(t['id'])
            release_date.append(t['release_date'])
            total_tracks.append(t['total_tracks'])
            try:
                url_image.append(t['images'][0]['url'])
            except IndexError:
                url_image.append('')

        # MAJ offset en fonction de la limite
        offset += limit

    # création DF avec les lignes récupérées
    df_album = pd.DataFrame({
        'artist_id': artist_sp_id,
        'artist_name' : artist_name,
        'album_name': album_name,
        'album_id': album_id,
        'release_date': release_date,
        'total_tracks': total_tracks,
        'url_image': url_image
    })

    print(f"{len(df_album)} albums récupérés pour {artist_id}\n")

    time.sleep(2)
    
    return df_album

In [ ]:
# Liste pour stocker les résultats
df_list = []

# Parcours des 10 premiers artistes
for i in range(200):
    print(f"Récupération des albums pour l'artiste {i+1} / 200")
    
    df_album = search_album_artist(data.artist_id[i])
    
    # Ajout du DataFrame de l'artiste à la liste
    df_list.append(df_album)

# Concaténation de tous les DataFrames
df_list_album = pd.concat(df_list, ignore_index=True)

Récupération des albums pour l'artiste 1 / 200
Récupération des albums pour l'artiste 2 / 200
Récupération des albums pour l'artiste 3 / 200
Récupération des albums pour l'artiste 4 / 200
Récupération des albums pour l'artiste 5 / 200
Récupération des albums pour l'artiste 6 / 200


### 3_ Requête infos artistes depuis liste artistes

In [ ]:
# on ouvre le fichier de artistes
data = pd.read_csv('liste_artistes.csv')

In [20]:
# récupération des données via l'api pour la partie Albums
# via cette API https://developer.spotify.com/documentation/web-api/reference/get-an-artist

# création d'une fonction récupérant le dataframe des artistes avec leurs id pour en récupérer les informations

# on utilise un dataframe en entrée qui doit contenir au moins deux colonnes. une avec les id artiste ('artist_id')
# et l'autre avec les nom d'artiste (artist_name) . on pourrait faire sans le second vu que le nom est dans l' API
# a voir si on le fait qu'avec une liste d'ID ? 
def search_infos_artist(df):  

    # nous avons besoin de listes pour récupérer les infos des artistes

    artist_name = []
    artist_id = []
    artist_popularity = []
    artist_followers = []
    artist_genres = []
    artist_image = [] 
    
    # on parcours la liste des id des artistes et grace au df on peut récupérer le nom.
    for ele, val in enumerate(df['artist_id']):
        # pour chaque artistes on lance une requête sur l'API pour récupérer les données
        artist_results = sp.artist(val)

        # on créé une liste des informations de l'liste_artistes
        artist_name.append(df['artist_name'][ele])
        artist_id.append(val)
        artist_popularity.append(artist_results['popularity'])
        artist_followers.append(artist_results['followers']['total'])
        artist_genres.append(artist_results['genres'])
        
        # certaines images vides, donc on passe par un try pour essayer de récupérer, sinon, on indique vide
        try:
            artist_image.append(artist_results['images'][0]['url'])
        except:
            artist_image.append('')

        
        # on patiente 1 seconde entre deux DF pour limiter l'accès à l'API
        time.sleep(1)
        print(ele," artiste sur, 1000")

    # # une fois les lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_artist = pd.DataFrame({'artist_name':artist_name,\
                                'artist_id' : artist_id,\
                                'popularity' : artist_popularity,\
                                'artist_followers' : artist_followers,\
                                'genres' : artist_genres,\
                                'image' : artist_image})

    # on retourne le df créé
    return df_artist

In [37]:
data3 = data3.reset_index(drop=True)

In [32]:
data3 = data2.loc[4680:4690]

In [38]:
df = search_infos_artist(data3)

0  artiste sur, 1000
1  artiste sur, 1000
2  artiste sur, 1000
3  artiste sur, 1000
4  artiste sur, 1000
5  artiste sur, 1000
6  artiste sur, 1000
7  artiste sur, 1000
8  artiste sur, 1000
9  artiste sur, 1000
10  artiste sur, 1000


In [27]:
data2 = pd.read_csv('df_list_artist_p2.csv')

In [29]:
data2[data2['artist_name'] == 'Marco Frisina']

,artist_name,artist_id
4689,Marco Frisina,0tEWpk8Do6YAmD78I60CR0


### 4_ Requête titres par albums (version par liste d'albums pour éviter le time-out, cf partie suivante)

In [21]:
# on conserve le df créé précédemment. ou on le récupère

df_list_album = pd.read_csv('list_artist_album.csv') 

# création de liste de 10 éléments pour l'API des titres. On va faire une liste de liste

liste_albums_global = list(df_list_album['album_id'])

# définition d'une fonction itérative utilisant le yield (Contrairement à return, yield ne termine pas la fonction ; 
# il suspend son exécution et reprend là où elle s’était arrêtée.)  cf : https://blog.alphorm.com/maitriser-yield-python

def creation_nb_liste(lst,nb):  # création d'une fonction qui prends deux arguments, une liste et un nombre. On utilisera ce nombre pour déterminer le nombre d'élément de la sous liste
    for ele in range(0,len(lst),nb): # on parcours la liste avec un pas du nombre nb
        yield lst[ele:ele + nb]  # a chaque itération on slicer la liste de nb valeurs pour créer une sous liste.

In [22]:
# on génére une liste du résultat de la fonction précédente
liste_album_par_dix = list(creation_nb_liste(liste_albums_global, 20))

In [23]:
# test de la fonction de création d'une liste de liste de 10 albums pour éviter le time-out
liste_album_par_dix

[['4R6FV9NSzhPihHR0h4pI93',
  '1x6p39FFIESOq8q8B4oQXN',
  '494aUGKXMH5ruXtR3O1a3H',
  '1X472EvsSqH09RyrqbtyXU',
  '3Q9wXhEAX7NYCPP0hxIuDz',
  '28DUZ0itKISf2sr6hlseMy',
  '0zcNtUZ2oEpRmiDuWBFvcX',
  '5Eevxp2BCbWq25ZdiXRwYd',
  '6hU9JCoqq4GjYq86dQ1o9b',
  '3XB2yloP7l00tEUmaODtVi',
  '2FUsvD1bw53HGOjAg56vRD',
  '7qRKpkP0jSOlBMLYZhAMCh',
  '4XHIjbhjRmqWlosjj5rqSI',
  '0hJ3A7sih1AXDoMjXHLr7Q',
  '7IYqppCBhR5z9z8JqgXuxi',
  '4flcwtqnLoKZJ2wrCp1aJq',
  '1FiRqhpAowNK8gTl5sOhxZ',
  '4fy0SXW5G8evBCo2A3kn02',
  '5uvXx5ZQswNRFCdHR521YZ',
  '72seWTJF9U5SljizfyF2ZK'],
 ['4JtIVJRA342O0YoAchen5Q',
  '7pgs38iLfEqUtwgCRgvbND',
  '0ZBE7rVC0zKFVt5osvXlnz',
  '0f7R0jf0pcTb6K6IVVPcMD',
  '2NbO8RRVTVEUjHaxUVdMDT',
  '6hPkbAV3ZXpGZBGUvL6jVM',
  '2lZ9RqGnGXH8gtH3DK02tJ',
  '2xYsmEzewmcKVPlYXOXVeh',
  '73SsqFGUL7Xns1ZK3weKiK',
  '1W990lM14o82dzCUGEtvTd',
  '6tbD7TXKlegmp644gE2iKe',
  '3hkyt2ofB7tGrsbxwBaCE9',
  '3MXrPupQnGD3XQLRbxDyW9',
  '3HCCUaRSjHSFOe4fqE0BiP',
  '5c6tiN4r4GwOTkuJKLzW2X',
  '3I34EprFC0rlAO4p

In [69]:
for ele in liste_album_par_dix:
    print(ele)

['4R6FV9NSzhPihHR0h4pI93', '1x6p39FFIESOq8q8B4oQXN', '494aUGKXMH5ruXtR3O1a3H', '1X472EvsSqH09RyrqbtyXU', '3Q9wXhEAX7NYCPP0hxIuDz', '28DUZ0itKISf2sr6hlseMy', '0zcNtUZ2oEpRmiDuWBFvcX', '5Eevxp2BCbWq25ZdiXRwYd', '6hU9JCoqq4GjYq86dQ1o9b', '3XB2yloP7l00tEUmaODtVi', '2FUsvD1bw53HGOjAg56vRD', '7qRKpkP0jSOlBMLYZhAMCh', '4XHIjbhjRmqWlosjj5rqSI', '0hJ3A7sih1AXDoMjXHLr7Q', '7IYqppCBhR5z9z8JqgXuxi', '4flcwtqnLoKZJ2wrCp1aJq', '1FiRqhpAowNK8gTl5sOhxZ', '4fy0SXW5G8evBCo2A3kn02', '5uvXx5ZQswNRFCdHR521YZ', '72seWTJF9U5SljizfyF2ZK']
['4JtIVJRA342O0YoAchen5Q', '7pgs38iLfEqUtwgCRgvbND', '0ZBE7rVC0zKFVt5osvXlnz', '0f7R0jf0pcTb6K6IVVPcMD', '2NbO8RRVTVEUjHaxUVdMDT', '6hPkbAV3ZXpGZBGUvL6jVM', '2lZ9RqGnGXH8gtH3DK02tJ', '2xYsmEzewmcKVPlYXOXVeh', '73SsqFGUL7Xns1ZK3weKiK', '1W990lM14o82dzCUGEtvTd', '6tbD7TXKlegmp644gE2iKe', '3hkyt2ofB7tGrsbxwBaCE9', '3MXrPupQnGD3XQLRbxDyW9', '3HCCUaRSjHSFOe4fqE0BiP', '5c6tiN4r4GwOTkuJKLzW2X', '3I34EprFC0rlAO4pKNeX1t', '57KFwKnC0ZjaIQGnhE7ops', '0VEFy5MsBiq0u2lWL0OwOd', '3tsXyEbUQ

In [25]:
# on récupère une liste simple de 20 valeur
liste_album_par_dix[0:1][0]

['4R6FV9NSzhPihHR0h4pI93',
 '1x6p39FFIESOq8q8B4oQXN',
 '494aUGKXMH5ruXtR3O1a3H',
 '1X472EvsSqH09RyrqbtyXU',
 '3Q9wXhEAX7NYCPP0hxIuDz',
 '28DUZ0itKISf2sr6hlseMy',
 '0zcNtUZ2oEpRmiDuWBFvcX',
 '5Eevxp2BCbWq25ZdiXRwYd',
 '6hU9JCoqq4GjYq86dQ1o9b',
 '3XB2yloP7l00tEUmaODtVi',
 '2FUsvD1bw53HGOjAg56vRD',
 '7qRKpkP0jSOlBMLYZhAMCh',
 '4XHIjbhjRmqWlosjj5rqSI',
 '0hJ3A7sih1AXDoMjXHLr7Q',
 '7IYqppCBhR5z9z8JqgXuxi',
 '4flcwtqnLoKZJ2wrCp1aJq',
 '1FiRqhpAowNK8gTl5sOhxZ',
 '4fy0SXW5G8evBCo2A3kn02',
 '5uvXx5ZQswNRFCdHR521YZ',
 '72seWTJF9U5SljizfyF2ZK']

### 5_ Récupération infos titres depuis liste albums

In [9]:
# récupération des données via l'api pour la partie titres

# nous avons besoin de listes pour récupérer les infos des albums
df_list_track_album = pd.DataFrame(columns=['artist_name','artist_id','album_id','track_id','disc_number','track_number','track_name','duration_ms'])

# création d'une fonction récupérant la list des artistes sur une années donnée en argument
def search_track_album(album_id):    
    
    # nous avons besoin de deux listes pour récupérer les noms et les id spotify que l'on réinitialise à chaque fois
    artist_name = []
    artist_id = []
    track_id = []
    disc_number = []
    track_number = []
    track_name = []
    duration_ms = []


    
    # on reste sur le même shéma que la fonction précédente même si on ne risque pas de se retrouver avec trop de page. 
    # je préfère anticiper le groupe qui aura plus de 50 albums et donc deux pages. 
    for i in range(0,1000,50):
        
        # on lance l'API pour récupérer les données de la page en cours avec l'année en paramètre (en str)
        albums_track_results = sp.album_tracks(album_id, limit=50,offset=i)
        
        # on intègre les données récupérées dans les bonnes listes
        # on parcours les différents albums de la page
        for i, t in enumerate(albums_track_results['items']):
            # pour chaque album on récupère le nom de l'artiste et son id Spotify (Plusieurs autres champs possibles)
            # récupéranles ici : https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks

            artist_name.append(t['artists'][0]['name'])
            artist_id.append(t['artists'][0]['id'])
            track_id.append(t['id'])
            disc_number.append(t['disc_number'])
            track_number.append(t['track_number'])
            track_name.append(t['name'])
            duration_ms.append(t['duration_ms'])# diviser par /60000 pour l'avoir en minutes
            
    
    # une fois les 1000 lignes récépérées on créé un DataFrame que l'on retourne en réponse à l'appel de cette fonction
    df_track_album = pd.DataFrame({'artist_name':artist_name,\
                             'artist_id' : artist_id,\
                             'album_id' : album_id,\
                             'track_id' : track_id,\
                             'disc_number' : disc_number,\
                             'track_number' : track_number,\
                             'track_name' : track_name,\
                             'duration_ms' : duration_ms})
    df_list_track_album2 = pd.concat([df_list_track_album, df_track_album])
    
    # on retourne le df créé
    return df_list_track_album2

In [ ]:
# le test fonctionne bien pour un album
test = search_track_album('1X472EvsSqH09RyrqbtyXU')
test

In [ ]:
# on récupère la liste des titres des 5 premiers albums de notre DF initial
for i in range(64):
    df_list_track_album = search_track_album(df_list_album.loc[i,'album_id'])
    # on ajoute une pause vu que l'API ne souhaite pas que l'on puisse faire trop de requêtes à la suite
    print(f"Albun n°{i} nommé {df_list_album.loc[i,'album_name']} vient d'être ajouté")
    time.sleep(1)  # 1 sec de pause entre deux requêtes
    

# TEST / Spotify

## TEST / recherche par la search avec album par artiste

In [6]:
# https://developer.spotify.com/documentation/web-api

# on utilise une librairie créé et dédiée à l'api Spotify
# pip install spotipy --upgrade  # pour installer ou upgrader la librairie

# information spotify à intégrer pour pouvoir travailler
# SPOTIPY_CLIENT_ID='5a63812d960f4789a86499ecb6073348'
# SPOTIPY_CLIENT_SECRET='31cf73ebc10247fcb28175c1a777f34a'
# SPOTIPY_REDIRECT_URI='http://localhost:8080'

# on importe cette librairie
import spotipy
from spotipy.oauth2 import SpotifyOAuth


# on créé une connexion avec les données du compte.
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='5a63812d960f4789a86499ecb6073348',

                                               client_secret='31cf73ebc10247fcb28175c1a777f34a',

                                               redirect_uri='http://localhost:8080',

                                               scope='user-library-read'))

# Fetch a track by its Spotify ID

track_id = '6rPO02ozF3bM7NnOV4h6s2'

track = sp.track(track_id)

print(track['name'], 'by', track['artists'][0]['name'])


Despacito - Remix by Luis Fonsi


In [26]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:1984', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])


In [27]:
import pandas as pd
track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'album_name' : track_name, 'released_date' : track_id})
print(track_dataframe.shape)
track_dataframe


(983, 3)


,artist_name,album_name,released_date
0,Radiohead,OK Computer,1997-05-28
1,Radiohead,In Rainbows,2007-12-28
2,Radiohead,Kid A,2000-10-02
3,Radiohead,The Bends,1995-03-13
4,Radiohead,Pablo Honey,1993-02-22
...,...,...,...
978,Jéf,Interior,2015-07-31
979,Radiophobic,wdk,2023-07-28
980,Radio Cult,Retroactive,2008-01-07
981,Detre Val,Radio,2023-05-13


## TEST / recherche par search avec albums par années ici 2024

In [ ]:

# on importe cette librairie
import spotipy
from spotipy.oauth2 import SpotifyOAuth


# on créé une connexion avec les données du compte.
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='5a63812d960f4789a86499ecb6073348',

                                               client_secret='31cf73ebc10247fcb28175c1a777f34a',

                                               redirect_uri='http://localhost:8080',

                                               scope='user-library-read'))


In [59]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2025', type='album', limit=50,offset=i)
    for i, t in enumerate(track_results['albums']['items']):
        artist_name.append(t['artists'][0]['name'])
        popularity.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['release_date'])

In [62]:
import pandas as pd
track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'id_artiste' : popularity, 'album_name' : track_name, 'released_date' : track_id})
print(track_dataframe.shape)
track_dataframe

(1000, 4)


,artist_name,id_artiste,album_name,released_date
0,De La Soul,1Z8ODXyhEBi3WynYw0Rya6,Bigger (feat. Choklate),2025-01-21
1,Amenra,0N1jE1EIrhZjvQSfuLupUu,Heden,2025-01-21
2,Bibi Babydoll,6R26kGib2ubbZJMh7kKVGV,"Mulher Não Trai, Mulher Se Vinga",2025-01-21
3,ZUTOMAYO,38WbKH6oKAZskBhqDFA8Uj,シェードの埃は延長,2025-01-21
4,Aitch,2PJEagPIxaBugeMjIyKVXF,Raving In The Studio (Instrumental / Radio Edit),2025-01-21
...,...,...,...,...
995,Qing Madi,0ZvsEkINadmEV4qzS4ollh,Akanchawa,2025-01-10
996,Mc Denny,7dUNKhz7HdxExteqcREGLi,Automotivo Vai Faz a Fila,2025-01-12
997,DJ MH,0P5X0nx1hQO7LSdmekF6xA,Melodia da Horizon,2025-01-10
998,Breno Major,53cEbokP6S7cahtXPXvQ5c,Envolvidão,2025-01-09


In [46]:
track_dataframe[track_dataframe['artist_name']== "Linkin Park"]

,artist_name,id_artiste,album_name,released_date
2,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,From Zero,2024-11-15
102,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,The Emptiness Machine,2024-09-05
302,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,Heavy Is the Crown,2024-09-24
615,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,Papercuts,2024-04-12
754,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,Over Each Other,2024-10-24


## TEST / autres test

In [63]:
track_dataframe.artist_name.value_counts()

artist_name
Mc Gw                      66
Johann Sebastian Bach      35
Ludwig van Beethoven       22
Yzalune                    22
Wolfgang Amadeus Mozart    21
                           ..
Bren Joy                    1
Mc Denny                    1
DJ MH                       1
Breno Major                 1
Øneheart                    1
Name: count, Length: 588, dtype: int64

In [10]:
# récupération d'info d'un titre

import requests

track_id = '6rPO02ozF3bM7NnOV4h6s2'

# Récupérer des informations sur une piste en Python
url = 'https://api.spotify.com/v1/tracks/{track_id}'
headers = {
    'Authorization': 'Bearer ACCESS_TOKEN'
}

response = requests.get(url, headers=headers)
track_info = response.json()
print(track_info)

{'error': {'status': 401, 'message': 'Invalid access token'}}


In [1]:
import pandas as pd    
jsonObj = pd.read_json(path_or_buf='recording', lines=True)

In [5]:
jsonObj

,title,isrcs,rating,tags,artist-credit,aliases,video,id,relations,disambiguation,length,genres,annotation
0,Suck,[],"{'value': None, 'votes-count': 0}",[],"[{'artist': {'genres': [{'disambiguation': '',...",[],False,10b9c34b-821d-4cc8-9587-5c7d0cb68865,"[{'source-credit': '', 'target-type': 'work', ...",,NaN,[],None
1,Whispering Shadows,[],"{'votes-count': 0, 'value': None}","[{'count': 2, 'name': 'death metal'}, {'name':...","[{'artist': {'name': 'Unanimated', 'tags': [{'...",[],False,f28b2394-bd5d-4f0c-a0de-55ce015a5c57,"[{'source-credit': '', 'target-type': 'work', ...",,NaN,[{'id': 'eacfa027-2fad-413f-a2f1-80fa43674f0b'...,None
2,Kiss-A-Me Baby,[],"{'value': None, 'votes-count': 0}",[],[{'artist': {'genres': [{'id': 'fe12b346-a10e-...,[],False,b26c15bd-9e01-4bc7-8bb0-63f58baa739b,"[{'begin': None, 'type-id': 'a3005666-a872-32c...",,NaN,[],None
3,The Calling,[],"{'value': None, 'votes-count': 0}",[],"[{'artist': {'tags': [{'count': 1, 'name': 'an...",[],False,565de6d8-f93f-4b44-97e2-c1f3ffc2410a,[],,264000.0,[],None
4,Mojo's Intro,[],"{'value': 1, 'votes-count': 1}",[],"[{'name': 'Forbidden Pigs', 'joinphrase': '', ...",[],False,17d3194b-c11d-4471-9938-829560e4383f,[],,242000.0,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131215,Joker,[GXBAV2397726],"{'votes-count': 0, 'value': None}",[],"[{'name': 'Ezequiel Medeiros', 'artist': {'typ...",[],False,f8b553b3-d28d-4fb8-a52a-aca6da567d01,"[{'source-credit': '', 'target-type': 'url', '...",,91000.0,[],None
131216,Shores,[],"{'votes-count': 0, 'value': None}",[],"[{'name': 'Zeni', 'artist': {'tags': [], 'name...",[],False,50c833ce-4cb1-47dc-a71f-be31b018eb8b,[],,362000.0,[],None
131217,Better Days,[],"{'value': None, 'votes-count': 0}",[],"[{'artist': {'type-id': None, 'aliases': [], '...",[],False,dd6b23a6-74b3-4308-bbb6-5d7d22158500,[],,288000.0,[],None
131218,Gay Fish,[],"{'votes-count': 0, 'value': None}",[],"[{'name': 'Trey Parker', 'joinphrase': '', 'ar...",[],True,753e9ff1-6e8a-4714-bf79-296b5612dd7c,"[{'begin': None, 'target-credit': '', 'type-id...",,84000.0,[],None
